<a href="https://colab.research.google.com/github/pyrated03/IISc-Incremental-Learning/blob/main/CUB_Data_Resnet_train(IISc)_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import gc
import numpy as np
import pandas as pd
import cv2
import os
import time
import os.path as osp
import pickle
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

class CUB2002011_ds(Dataset):
    def __init__(self, data_dir, mode="train"):
        assert mode in ["train", "test"] #Make sure the mode is one of the two
        super(CUB2002011_ds, self).__init__() #Inheritance dataset __init__()
        train_test_split = pd.read_csv(
            f"{data_dir}/train_test_split.txt",
            sep=" ",
            names=["img_id", "is_train"],
        )
        img_paths = pd.read_csv(
            f"{data_dir}/images.txt", sep=" ", names=["img_id", "img_path"]
        )  #Read all image paths
        labels = pd.read_csv(
            f"{data_dir}/image_class_labels.txt",
            sep=" ",
            names=["img_id", "img_label"],
        )
        self.data_dir = osp.join(data_dir, "images")
        self.mode = 1 if mode == "train" else 0  #Choose whether it is 1 training mode or test mode 0
        self.img_path = img_paths.loc[
            train_test_split["is_train"] == self.mode, "img_path"
        ].values  #Get the corresponding path in the corresponding mode
        self.label = labels.loc[
            train_test_split["is_train"] == self.mode, "img_label"
        ].values #Get the label in the corresponding mode
        ############################################################################
        split_idx = len([x for x in self.label if x<=100])
        print(split_idx)
        self.img_path = self.img_path[:split_idx]
        self.label = self.label[:split_idx] 
        ############################################################################

        img_sz = 84
        if mode == 'test':
            self.transform = transforms.Compose([
                transforms.Resize((img_sz, img_sz)),
                transforms.ToTensor()
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((img_sz, img_sz)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(30),
                transforms.ToTensor()
            ])
        self.y = self.label

    def __len__(self):
        return self.label.shape[0] #The size of the entire data set in the corresponding mode

    def __getitem__(self, idx):
        img = self.transform(
            Image.open(f"{self.data_dir}/{self.img_path[idx]}").convert("RGB")
        ) #Open the path of the image in the corresponding folder of the data set and convert the image data to RGB
        label = self.label[idx]
        # if (int)(label) <= 100:
        return img, label-1
        # else:
        #   return

In [ ]:
import random

data_dir = "gdrive/My Drive/IITB Internship/CUB_200_2011"
train_dset = CUB2002011_ds(data_dir = data_dir, mode = "train")
batch_size = 16
  
# l = len(train_dset)
# split = int(np.floor(0.9 * l))
# val_dset = train_dset[split:]
# train_dset = train_dset[:split]

size_ds = len(train_dset)
indices = list(range(size_ds))
split = int(np.floor(0.9 * size_ds))
random.shuffle(indices)
train_sampler = indices[:split]
val_sampler = indices[split:]

train_ds = DataLoader(train_dset, batch_size=batch_size, sampler=train_sampler, drop_last=True) #shuffle = True

val_ds = DataLoader(train_dset, batch_size=batch_size, sampler=val_sampler, drop_last=True)#shuffle = True
data = {'train':train_ds,'val':val_ds}



3000


In [ ]:
test_dset = CUB2002011_ds(data_dir = data_dir, mode = "test")
test_ds = DataLoader(test_dset, batch_size=batch_size, drop_last=True)#shuffle = True)

2864


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [ ]:
dataset_sizes = {'train':batch_size*len(train_ds), 'val':batch_size*len(val_ds),'test':batch_size*len(test_ds)}
dataset_sizes

{'test': 2864, 'train': 2688, 'val': 288}

In [ ]:
data['train']

In [ ]:
import torchvision.models as models

model = models.resnet18(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
model.fc = nn.Sequential(nn.Linear(512,100))#, nn.Softmax(93))
# model
model = model.to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
def train_model(model, criterion, softmax, optimizer, scheduler, device, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in data[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # outputs = softmax(outputs)
                    _, preds = torch.max(outputs, 1)
                    # outputs = softmax(outputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # print("pred: ",preds) 
                # print("label: ", labels.data)
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                # running_corrects += torch.sum(preds == labels)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / (dataset_sizes[phase])
            epoch_acc = running_corrects*1.0 / (dataset_sizes[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# def train_model(model, criterion, softmax, optimizer, scheduler, device, num_epochs=5):
#     since = time.time()

#     best_model_wts = copy.deepcopy(model.state_dict())
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

#             running_loss = 0.0
#             running_corrects = 0

#             # Iterate over data.
#             if phase == 'train':
#               for inputs, labels in train_ds:
#                   inputs = inputs.to(device)
#                   labels = labels.to(device)

#                   # zero the parameter gradients
#                   optimizer.zero_grad()

#                   # forward
#                   # track history if only in train
#                   with torch.set_grad_enabled(phase == 'train'):
#                       outputs = model(inputs)
#                       _, preds = torch.max(outputs, 1)
#                       loss = criterion(outputs, labels)

#                       # backward + optimize only if in training phase
#                       if phase == 'train':
#                           loss.backward()
#                           optimizer.step()

#                   # statistics
#                   running_loss += loss.item() * inputs.size(0)
#                   running_corrects += torch.sum(preds == labels.data)
#               if phase == 'train':
#                   scheduler.step()

#               epoch_loss = running_loss / dataset_sizes[phase]
#               epoch_acc = running_corrects.double() / dataset_sizes[phase]

#               print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                   phase, epoch_loss, epoch_acc))
              
# ##################################################################################################################################
#             if phase == 'val':
#               for inputs, labels in val_ds:
#                   inputs = inputs.to(device)
#                   labels = labels.to(device)

#                   # zero the parameter gradients
#                   optimizer.zero_grad()

#                   # forward
#                   # track history if only in train
#                   with torch.set_grad_enabled(phase == 'train'):
#                       outputs = model(inputs)
#                       _, preds = torch.max(outputs, 1)
#                       loss = criterion(outputs, labels)

#                       # backward + optimize only if in training phase
#                       if phase == 'train':
#                           loss.backward()
#                           optimizer.step()

#                   # statistics
#                   running_loss += loss.item() * inputs.size(0)
#                   running_corrects += torch.sum(preds == labels.data)
#               if phase == 'train':
#                   scheduler.step()

#               epoch_loss = running_loss / dataset_sizes[phase]
#               epoch_acc = running_corrects.double() / dataset_sizes[phase]

#               print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                   phase, epoch_loss, epoch_acc))

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

#         print()

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

#     # load best model weights
#     model.load_state_dict(best_model_wts)
#     return model

In [ ]:
criteria = nn.CrossEntropyLoss()
softmax = nn.Softmax()
# Observe that all parameters are being optimized
# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
# Number of epochs
eps=75

# Adam Optimizer(75 epochs)

In [ ]:
model = train_model(model, criteria, softmax, optimizer, scheduler, device, eps)

Epoch 0/74
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 4.3856 Acc: 0.0632
val Loss: 3.8281 Acc: 0.1181

Epoch 1/74
----------
train Loss: 3.2617 Acc: 0.2374
val Loss: 3.1009 Acc: 0.2153

Epoch 2/74
----------
train Loss: 2.6018 Acc: 0.3765
val Loss: 2.7791 Acc: 0.2569

Epoch 3/74
----------
train Loss: 2.1342 Acc: 0.4762
val Loss: 2.6319 Acc: 0.3090

Epoch 4/74
----------
train Loss: 1.7475 Acc: 0.5759
val Loss: 2.5840 Acc: 0.2951

Epoch 5/74
----------
train Loss: 1.4335 Acc: 0.6577
val Loss: 2.4578 Acc: 0.3264

Epoch 6/74
----------
train Loss: 1.1626 Acc: 0.7266
val Loss: 2.4778 Acc: 0.3750

Epoch 7/74
----------
train Loss: 0.9580 Acc: 0.7827
val Loss: 2.4228 Acc: 0.3611

Epoch 8/74
----------
train Loss: 0.7295 Acc: 0.8571
val Loss: 2.3983 Acc: 0.4062

Epoch 9/74
----------
train Loss: 0.6079 Acc: 0.8880
val Loss: 2.4801 Acc: 0.3715

Epoch 10/74
----------
train Loss: 0.4444 Acc: 0.9308
val Loss: 2.5347 Acc: 0.3403

Epoch 11/74
----------
train Loss: 0.3586 Acc: 0.9453
val Loss: 2.6065 Acc: 0.3681

Epoch 12/74
----------
t

In [ ]:
running_corrects_test = 0
phase = "test"
for input_test_final, label_test_final in test_ds:
    input_test_final = input_test_final.float().to(device)
    label_test_final = label_test_final.to(device)
    # print("validation:",iter_valid)
    outputs_test_final = model(input_test_final)
    _, preds_test_final = torch.max(outputs_test_final, 1)
    running_corrects_test += torch.sum(preds_test_final == label_test_final.data)

epoch_acc_test = running_corrects_test*1.0 / (dataset_sizes[phase])
print(epoch_acc_test)

tensor(0.4417, device='cuda:0')
